# 点云配准 (Registration)
### 1. 点云变换（i.e. transformation）

In [ ]:
# 1.1 平移算法
'''
理论基础：
    将P沿着向量d方向移动得到P':
        P' = P + d, 
        s.t. P' = (x',y',z',1), P = (x,y,z,1), d = (a,b,c,1)
    其中:
        x' = x + a, y' = y + b, z' = z + c
    将转换过程表示为: P' = TP，其变换矩阵T为:
        T = T(a,b,c) = (1 0 0 a
                        0 1 0 b
                        0 0 1 c
                        0 0 0 1)    
代码实现：
    通过三维向量t来平移所有点 s.t. v_t = v + t
'''
import copy

mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()    #生成坐标轴模型
mesh_tx = copy.deepcopy(mesh).translate((1.3,0,0))
mesh_tx.paint_uniform_color([0, 1, 0])                        # 沿X方向平移的，渲染为绿色
mesh_ty = copy.deepcopy(mesh).translate((0,1.3,0))
mesh_ty.paint_uniform_color([1, 0, 0])                        # 沿Y方向平移的，渲染为红色

print(f'Center of mesh: {mesh.get_center()}')
print(f'Center of mesh tx: {mesh_tx.get_center()}')
print(f'Center of mesh ty: {mesh_ty.get_center()}')

o3d.visualization.draw_geometries([mesh, mesh_tx, mesh_ty])

'''
重要函数：
    translate()
输入参数：
    (x,y,z)                   # 目标坐标
    relative = True           # True 则在从中心点的位置上位移(x,y,z)
                              # False 则将中心点位移至(x,y,z)

重要函数：
    deepcopy()
输入参数：
    x                         # 返回一个x的deepcopy (i.e. 复制object和class)
'''

In [ ]:
# 1.2 旋转
'''
理论基础：
    旋转取决于旋转轴与绕旋转轴转动的角度。考虑(x,y)绕z轴旋转，若(x,y)的极坐标为：
        x = r*cos(a)
        y = r*sin(a)
    旋转角度记为b，则新的位置(x',y')有以下坐标：
        x' = r*cos(a+b)
        y' = r*sin(a+b)
    展开得(复合角公式)：
        x' = x*cos(b) - y*sin(b)
        y' = x*sin(b) + y*cos(b)
    或者以矩阵形式：
        (x'; y') = (cos(b) -sin(b); sin(b) -cos(b)) (x; y)
    因此分别绕z,x,y旋转的旋转矩阵（以(x; y; z; 1)向量的格式）为：
        Rz = (cos(b) -sin(b) 0 0
              sin(b) -cos(b) 0 0
              0      0       1 0
              0      0       0 1)
              
        Rx = (1 0      0       0
              0 cos(b) -sin(b) 0
              0 sin(b) cos(b)  0
              0 0      0       1)  
              
        Rz = (cos(b)  0 sin(b) 0
              0       1 0      1
              -sin(b) 0 cos(b)
              0       0 0      1)       
'''
import copy

mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()     # 生成坐标轴
mesh_r = copy.deepcopy(mesh)
R = mesh.get_rotation_matrix_from_xyz((np.pi/2,0,np.pi/4))     # 欧拉角转变换矩阵
mesh_r.rotate(R, center=(0,0,0))
mesh_r.paint_uniform_color([1, 0.75, 0])                       # 旋转后的渲染为金色
o3d.visualization.draw_geometries([mesh, mesh_r])

'''
重要函数：
    rotate()
输入参数：
    R                    # 旋转矩阵
    center = (x,y,z)     # 绕着(x,y,z)旋转
                            若(x,y,z)为之前的中心点，则原地旋转
'''

In [ ]:
# 1.3 缩放
'''
理论基础：
    缩放可以独立应用于三个坐标轴，如将点(x,y,z)转化成新的点：
        x' = ax, y' = by, z' = cz
    则缩放矩阵为：
        S = S(a,b,c) = (a 0 0 0
                        0 b 0 0
                        0 0 c 0
                        0 0 0 1)
    注：如果a=b=c，则同比例放大，否则发生变形
'''
# 代码实现：
import copy

mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()    # 生成坐标轴
mesh_s = copy.deepcopy(mesh).translate((2,0,0))               # 先位移坐标轴
mesh_s.scale(0.5, center=mesh_s.get_center())                 # 对其缩放
o3d.visualization.draw_geometries([mesh, mesh_s])

'''
重要函数：
    scale()
输入参数：
    x                     # 缩放程度
    center = (x,y,z)      # 在(x,y,z)这个点进行缩放
'''

In [ ]:
# 1.4 变换矩阵（General Transformation）
'''
变换矩阵含义：
T_P = [p11 p12 p13 p14
       p21 p22 p23 p24
       p31 p32 p33 p34
       p41 p42 p43 p44]
其中：
    旋转矩阵：
        [p11 p12 p13
         p21 p22 p23
         p31 p32 p33]
    平移向量：
        [p14
         p24
         p34]
    透视投影向量
        [p41 p42 p43]
    总体比例值
        [p44]
'''

# 代码实现
import copy

mesh = o3d.io.read_point_cloud("rect_prism.pcd")
mesh.paint_uniform_color([1, 0, 0])
T = np.eye(4)                                                 # 返回一个4x4的diagonal是1其余是0的矩阵
R = mesh.get_rotation_matrix_from_xyz((0,np.pi/3, np.pi/2))   # 生成旋转矩阵
print('旋转矩阵为：\n',R)

t = [0.1,0.1,0.2]
print('平移向量为：\n',t)
T[:3,:3] = R
T[:3,3] = t
T[3,3] = 1                                                    # 实际上是p44（计数从0开始）
print('变换矩阵为：\n',T)
mesh_t = copy.deepcopy(mesh).transform(T)
mesh_t.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([mesh, mesh_t])

### 2. 四元数、欧拉角、旋转向量转换为旋转矩阵

In [ ]:
# 2.1 四元数转旋转矩阵
'''
定义：
    在复数的基础上创造了四元数，以（a + bi + cj + dk）的形式说明空间点所在位置。
    其中i、j、k的几何意义可以理解为一种旋转。
转换原理：
    已知单位四元数[q0,q1,q2,q3]^T，则可计算得到旋转矩阵R:
        R(q^R) = [R11 R12 R13
                  R21 R22 R23
                  R31 R32 R33]
    其中：
        R11 = q0^2 + q1^2 - q2^2 - q3^2
        R12 = 2(q1q2 - q0q3)
        R13 = 2(q1q3 - q0q2)
        R21 = 2(q1q2 + q0q3)
        R22 = q0^2 - q1^2 + q2^2 - q3^2
        R23 = 2(q2q3 - q0q1)
        R31 = 2(q1q3 - q0q2)
        R32 = 2(q2q3 + q0q1)
        R33 = q0^2 - q1^2 - q2^2 + q3^2
'''

# 代码实现
R = o3d.geometry.get_rotation_matrix_from_quaternion((1, 0, 0, 2))     # 四元数转旋转矩阵，四元数的赋值顺序为：w、x、y、z
print('四元数转旋转矩阵；\n', R)

'''
主要函数：
    get_rotation_matrix_from_quaternion()
输入参数：
    q,x,y,z                                # 按此顺序进行赋值（不要求单位四元数）
'''


In [ ]:
# 2.2 欧拉角转旋转矩阵
'''
理论基础：
    当我们看到一个旋转矩阵时，很难想象具体的旋转方式。欧拉角则是把一次旋转分解成三次
    绕不同坐标轴的旋转。按照分解方式(e.g. x-y-z的顺序)，欧拉角的定义不一。
'''
# 代码实现
R1 = o3d.geometry.get_rotation_matrix_from_xyz((np.pi/2, 0, np.pi/4))   # 欧拉角转变换矩阵
print('沿X轴旋转90度，沿Z轴旋转45度的欧拉角转换成旋转矩阵为\n', R1)

'''
主要函数：
    get_rotation_matrix_from_xyz()         # 按x-y-z的顺序；同样也有_xzy,_yxz,_yzx...
输入参数：
    x,y,z                                  # 按给定顺序输入每个坐标轴的旋转角度
'''

In [ ]:
# 2.3 旋转向量转旋转矩阵
'''
理论基础：
    任何旋转都可以用一个旋转轴和旋转角来刻画。于是我们定义一个旋转向量，其方向与旋转轴一致，而长度等于旋转角。
'''

# 代码实现：
R = o3d.geometry.get_rotation_matrix_from_axis_angle((0, 0, np.pi/4))  # 旋转向量转旋转矩阵
print('沿Z轴旋转45度的旋转向量转换成旋转矩阵为；\n', R)

'''
重要函数：
    get_rotation_matrix_from_axis_angle()          # 实现旋转向量到旋转矩阵的转换
'''

# View坐标系和视图变换

In [ ]:
# 很详细的坐标转换讲解： https://blog.csdn.net/weixin_45590473/article/details/122884112?spm=1001.2101.3001.6650.5&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7ERate-5-122884112-blog-110096514.235%5Ev38%5Epc_relevant_anti_vip&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7ERate-5-122884112-blog-110096514.235%5Ev38%5Epc_relevant_anti_vip&utm_relevant_index=10

In [ ]:
import numpy as np
import open3d as o3d

In [ ]:
def euler_angle_to_RM(u,v,w):
    
    '''
目标：
    根据机械臂欧拉角计算旋转矩阵
    
输入参数：
    u,v,w 欧拉角（机械臂按x-y-z轴旋转顺序，旋转矩阵按Rz * Ry * Rx的顺序）
    
返回：
    3x3旋转矩阵
    '''

    # 角度转换弧度
    U = u/180*np.pi
    V = v/180*np.pi
    W = w/180*np.pi
    
    # 绕x轴旋转矩阵
    Rx = np.eye(3)                           # diagonal是1, 其余是0的矩阵
    Rx[1,1] = math.cos(U)
    Rx[1,2] = - math.sin(U)
    Rx[2,1] = math.sin(U)
    Rx[2,2] = math.cos(U)
    
    # 绕y轴旋转矩阵
    Ry = np.eye(3)
    Ry[0,0] = math.cos(V)
    Ry[0,2] = math.sin(V)
    Ry[2,0] = - math.sin(V)
    Ry[2,2] = math.cos(V)
    
    # 绕z轴旋转矩阵
    Rz = np.eye(3)
    Rz[0,0] = math.cos(W)
    Rz[0,1] = - math.sin(W)
    Rz[1,0] = math.sin(W)
    Rz[1,1] = math.cos(W)
    
    # 旋转矩阵 RM = Rz * Ry * Rx
    Imd = np.matmul(Rz, Ry)
    RM = np.matmul(Imd, Rx)

    return RM

In [ ]:
def compute_TM(pose1, pose2):
    '''
目标：
    计算一个将pose1的视角变到pose2的变换矩阵
    
原理：
    点云中的点在相机坐标系，而相机挂在机械臂上。因此只要把两个视角在机械臂坐标下统一，相机坐标系中自然也一致。
    首先分别计算生成pose1和pose2视角的变换矩阵(H1,H2)，然后根据矩阵乘法算出来相差的变换矩阵，用这个相差的
    变换矩阵将pose1转换到pose2的视角。
        i.e. T * H1 = H2
             T = H2 * H1^(-1)
             
参数：
    pose1: [x1,y1,z1,u1,v1,w1]           # 六个元素的列表
    pose2: [x2,y2,z2,u2,v2,w2]           # 机械臂坐标系下相机位置: (x1,y1,z1)
                                             # 单位：m
                                         # 相机的旋转欧拉角: (u1,v1,w1)
                                             # 单位：rad
    
返回：
    4 * 4 的变换矩阵，用于将pose1视角变换到和pose2相同
    '''
    
    # 计算pose1的变换矩阵
    T_P1 = np.eye(4)
    T_P1[:3,:3] = euler_angle_to_RM(pose1[3],   # 对应u1
                                    pose1[4],   # 对应v1
                                    pose1[5])   # 对应w1
    T_P1[:3,3] = pose1[:3]                      # 平移向量（单位：m）

    # 打印pose1的变换矩阵
#     print('pose1的变换矩阵: \n', T_P1)

    # 计算pose2的变换矩阵
    T_P2 = np.eye(4)
    T_P2[:3,:3] = euler_angle_to_RM(pose2[3],   # 对应u2
                                    pose2[4],   # 对应v2
                                    pose2[5])   # 对应w2
    T_P2[:3,3] = pose2[:3]                      # 平移向量（单位：m）

    # 打印pose2的变换矩阵
#     print('pose2的变换矩阵: \n', T_P2)

    # 计算从pose1变换到pose2的变换矩阵
    T_P1_inv = np.linalg.inv(T_P1)              # 计算pose1变换矩阵的逆矩阵 H1^(-1)
    TM = np.matmul(T_P2, T_P1_inv)              # T = H2 * H1^(-1)

    # 打印从pose1变换到pose2的变换矩阵
    print('点云之间的变换矩阵: \n', TM)

    return TM

In [ ]:
# 将两个点云在世界坐标系下统一

# PC1 机械臂视角:
x1 = 402.155 / 1000
y1 = -311.220 / 1000
z1 = 424.103 / 1000
u1 = 146.144
v1 = 10.344
w1 = 72.871

pose1 = [x1,y1,z1,u1,v1,w1]

# PC2 机械臂视角:
x2 = 349.246 / 1000
y2 = -329.168 / 1000
z2 = 428.859 / 1000
u2 = 140.927
v2 = 10.024
w2 = 108.486

pose2 = [x2,y2,z2,u2,v2,w2]

# PC3 机械臂视角:
x3 = 462.260 / 1000
y3 = -392.686 / 1000
z3 = 428.881 / 1000
u3 = 146.003
v3 = 3.532
w3 = -11.611

pose3 = [x3,y3,z3,u3,v3,w3]

# PC4 机械臂视角：
x4 = 455.619 / 1000
y4 = -421.647 / 1000
z4 = 428.845 / 1000
u4 = 145.485
v4 = 3.535
w4 = -57.437

pose4 = [x4,y4,z4,u4,v4,w4]

In [ ]:
# 将另外三个点云转换到pc2的视角

TM_1to2 = compute_TM(pose1,pose2)        # pose1转换到pose2的变换矩阵
TM_3to2 = compute_TM(pose3,pose2)        # pose3转换到pose2的变换矩阵
TM_4to2 = compute_TM(pose4,pose2)        # pose4转换到pose2的变换矩阵

In [ ]:
# 分别阅读四个点云
mesh1 = o3d.io.read_point_cloud('/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Block_pc/block1.pcd')
mesh2 = o3d.io.read_point_cloud('/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Block_pc/block2.pcd')
mesh3 = o3d.io.read_point_cloud('/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Block_pc/block3.pcd')
mesh4 = o3d.io.read_point_cloud('/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Block_pc/block4.pcd')

# 将pc1, pc3, pc4转换到pc2的视角
mesh2.transform(TM_1to2)
mesh3.transform(TM_3to2)
mesh4.transform(TM_4to2)

# 拼接结果可视化
o3d.visualization.draw_geometries([mesh1, mesh2, mesh3, mesh4])

# 根据两个向量计算旋转矩阵
### 罗德里格旋转公式

In [ ]:
'''
目标：
    给定的两个空间向量，计算出从一个向量旋转到另一个向量的旋转矩阵
    
原理：
    假设两个向量分别为v1 = (x1, y1, z1), v2 = (x2, y2, z2) -- 用np.asarray([])
    我们试图将v1旋转至v2
        1. va = normalize(v1), vb normalize(v2)    # 将两个向量转换为单位向量
        2. vs = vb x va                            # 叉乘得到旋转轴vs
        3. v = normalize(vs)                       # vs转为单位向量v
        4. ca = vb · va                            # 点乘得到旋转角cos值 (cos_angle)
        5. vt = v * scale, s.t. scale = 1 - ca     # 对v进行缩放，方便后面计算
        6. 旋转矩阵RM为3x3的矩阵：
            RM[0,0] = vt.x * v.x + ca
            RM[1,1] = vt.y * v.y + ca
            RM[2,2] = vt.z * v.z + ca
            
            vt.x = vt.x * v.y
            vt.z = vt.z * v.x
            vt.y = vt.y * v.z
            
            RM[0,1] = vt.x - vs.z
            RM[0,2] = vt.z - vs.y
            RM[1,0] = vt.x - vs.z
            RM[1,2] = vt.y - vs.x
            RM[2,0] = vt.z - vs.y
            RM[2,1] = vt.y - vs.x
            
参数：
    v1: 被旋转的向量，格式numpy 3x3 array
    v2: 要旋转到的目标向量，格式numpy 3x3 array
    
返回值：
    返回旋转矩阵RM
'''
def find_rm(v1,v2):
    # normalize v1 & v2
    va = v1 / np.sqrt(np.sum(v1**2))
    vb = v2 / np.sqrt(np.sum(v2**2))
    
    # vb cross va
    vs = np.cross(vb, va)
    
    # normalize vs
    v = vs / np.sqrt(np.sum(vs**2))
    
    # vb dot va & calculate scale
    ca = np.dot(vb,va)
    scale = 1 - ca
    vt = v * scale
    
    # 计算旋转矩阵
    RM = np.eye(3)
    RM[0,0] = vt[0] * v[0] + ca
    RM[1,1] = vt[1] * v[1] + ca
    RM[2,2] = vt[2] * v[2] + ca

    vt[0] = vt[0] * v[1]
    vt[2] = vt[2] * v[0]
    vt[1] = vt[1] * v[2]

    RM[0,1] = vt[0] - vs[2]
    RM[0,2] = vt[2] - vs[1]
    RM[1,0] = vt[0] - vs[2]
    RM[1,2] = vt[1] - vs[0]
    RM[2,0] = vt[2] - vs[1]
    RM[2,1] = vt[1] - vs[0]
    
    return RM 

In [ ]:
# 实验
v1 = np.asarray([1,0,0])
v2 = np.asarray([0,1,0])

find_rm(v1,v2)